In [1]:
import psycopg2
import os
import csv
import pandas as pd
import re

def chemin_fichier()-> str :
    """
        Donne le nom du chemin complet du fichier actuel (.py) 
    """
    chemin=os.getcwd() #methode pour tourner avec jupyter
    #chemin= path.dirname(path.abspath(__file__)) #methode d'Alice avec <os>
    #chemin = str(Path(__file__).parent) #ma methode avec <path>
    #print(chemin)
    return chemin

# etablir une connection
def connexion_DB(fichierDB):
    try:
        conn = psycopg2.connect(user = "santa",
                                password = "claus",
                                host = "localhost",
                                port = "5432:5432",
                                database = fichierDB
                                )

        cur = conn.cursor()
        # Print PostgreSQL Connection properties
        #print ( "Proprietes : ",conn.get_dsn_parameters(),"\n")

        # Print PostgreSQL version
        cur.execute("SELECT version();")
        record = cur.fetchone()
        #print("Version de PostgreSQL : ", record,"\n")
        cur.close()
    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)
    
    return conn

def deconnexion_DB(conn):
#closing database connection.
    if(conn):
        conn.close()
        print("PostgreSQL connection is closed")


In [3]:
# visualisation des fichiers a envoyer
conn=connexion_DB("santa_data")
cursor = conn.cursor()

rq_select='''
          SELECT 
              ID_l, Numero, Contenu FROM T_complete 
          ORDER BY id_l;
          '''

cursor.execute(rq_select)
res=cursor.fetchall()
print('id_l'.ljust(8),'Numero'.ljust(10),'Contenu'.ljust(15))
for row in res:
    print(str(row[0]).ljust(8),end='')
    print(str(row[1]).ljust(10),end='')
    print(str(row[2])[0:30].ljust(15))
    
cursor.close()
deconnexion_DB(conn)

id_l     Numero     Contenu        
1       64800     Bonjour Père Noël ! Je viens d
2       121741    Bonjour père Noël, Comment vas
3       98538     Bonjour Père Noël,J'ai était t
4       64687     Pere noël je veux une voiture 
5       64756     Je voudrai barbie avec un chev
6       121472    Bonjour Père Noël, J'espère qu
7       64890     Cher Père Noël,il est peut-êtr
8       66212     Pére noël quand tu viendras ch
9       117380    Cher Père Noël,noël approche, 
10      64709     Bonjour Père Noël je t aimes f
11      64573     Bonjour Père Noël, je t'aime b
12      121524    Cher Père Noël ,je suis heureu
13      121642    Bonjour Père Noël, je m'appell
14      65062     Père Noël JE VOUS ECRIS CETTE 
15      64516     Père noël, merci de ne me donn
16      44950     Cher petit papa noël,j'ai été 
17      121291    Nous t 'aimons beaucoup! Merci
18      104384    Coucou Père Noël. comment alle
19      107629    Je voudrai un camion un tracte
20      64736     Bonjour papa no

776     118368    Je veux qu il y ai beaucoup de
777     98389     Bonjour Père Noël, J'ai bientô
778     64556     Bonjour Père Noël,J'écris cett
779     127183    Cher Père Noël, ce que je veux
780     128365    Je veux un amoureux pour noël.
781     104340    Cher Père NoëlJe m'appelle Dam
782     130830    Pere noëlcette année j'ai été 
783     99094     Cher Père Noël,Bonjour Père No
784     64715     Cher Père Noël,j'ai été très s
785     65032     Bonjour Père Noël nous t'avons
786     127159    Bonjour Père Noël,J'ai très hâ
787     98523     Coucou Père Noël,Je suis conte
788     130110    Bisous gentil Père Noël je ne 
789     127078    BONJOUR Père NoëlAVEC MA MAMAN
790     102852    BONJOUR Père Noël CEST ALYSSON
791     36588     Cher Père Noëlje souaite que l
792     64436     Coucou je m'apelle sarah j'ai 
793     109157    Bonjour mon petit papa noël je
794     128621    Cher Père Noëlje m'appelle léo
795     64509     Cher Père Noël,Je suis très sa
796     65019     Bo

1397    64442     Bonjour mon papa noël,C'est ma
1398    64423     Merci pour tes cadeaux.
1399    99787     Chere Père Noëlje suis content
1400    65087     BONJOURJE T'ECRIS CETTE PETITE
1401    123792    Chèr père Noël je suis une fil
1402    101141    Pére noël je suis très gentil.
1403    64489     Bonjour papa noël,je te fais p
1404    101251    PÈRE   NOEL   JE   TE   FAIS  
1405    122696    Bonjour cher Père NoëlJ'ai été
1406    66305     Cher Père Noël.Je voudrais un 
1407    98335     Cher Papa Noêl,Je t'aime beauc
1408    121071    Coucou Père Noël,Le dimanche 1
1409    119097    Chère papa noèlj'ai bien ètè s
1410    114417    Père Noël, je suis très sage, 
1411    104385    Je souhaite avoir un beau cade
1412    115771    Bonjour ses vanessa je suis tr
1413    64786     Bonjour Père Noël,Je suis très
1414    101024    Cher Père Noël, je suis une pe
1415    101886    Cher papa noëlje suis impatien
1416    104471    Je te souhaite un joyeux noël 
1417    64673     Bonjour P

In [16]:
# insertion de id_l,contenu et numero vers T_contenus
conn=connexion_DB("santa_data")
cursor = conn.cursor()

rq_select='''INSERT INTO T_contenus(id_l, Numero, Contenu)
            SELECT id_l, Numero, Contenu
            FROM T_complete ORDER BY id_l
            ;'''
    
cursor.execute(rq_select)
conn.commit()

cursor.close()
deconnexion_DB(conn)

PostgreSQL connection is closed


In [17]:
# verification des données transmise
conn=connexion_DB("santa_data")
cursor = conn.cursor()

rq_select='''
          SELECT 
              id_c , ID_l, Numero, Contenu FROM T_Contenus 
          ORDER BY id_l;
          '''

cursor.execute(rq_select)
res=cursor.fetchall()
print('id_c'.ljust(8), 'id_l'.ljust(8),'Numero'.ljust(10),'Contenu'.ljust(15))
for row in res:
    print(str(row[0]).ljust(8),end='')
    print(str(row[1]).ljust(8),end='')
    print(str(row[2]).ljust(10),end='')
    print(str(row[3])[0:30].ljust(15))
    
cursor.close()
deconnexion_DB(conn)

id_c     id_l     Numero     Contenu        
1       1       64800     Bonjour Père Noël ! Je viens d
2       2       121741    Bonjour père Noël, Comment vas
3       3       98538     Bonjour Père Noël,J'ai était t
4       4       64687     Pere noël je veux une voiture 
5       5       64756     Je voudrai barbie avec un chev
6       6       121472    Bonjour Père Noël, J'espère qu
7       7       64890     Cher Père Noël,il est peut-êtr
8       8       66212     Pére noël quand tu viendras ch
9       9       117380    Cher Père Noël,noël approche, 
10      10      64709     Bonjour Père Noël je t aimes f
11      11      64573     Bonjour Père Noël, je t'aime b
12      12      121524    Cher Père Noël ,je suis heureu
13      13      121642    Bonjour Père Noël, je m'appell
14      14      65062     Père Noël JE VOUS ECRIS CETTE 
15      15      64516     Père noël, merci de ne me donn
16      16      44950     Cher petit papa noël,j'ai été 
17      17      121291    Nous t 'aimons be

580     580     64612     Cher Père Noël, comme cadeaux,
581     581     114471    Bonjour Père Noël,Cette année,
582     582     64444     Bonjour papa noël,tu sais j'ai
583     583     113175    Cher Père-Noël...Tout ce que j
584     584     124941    Cher père-noel je t'aime je vo
585     585     99687     J'aimerais que tu ma m'amenes 
586     586     124925    Cher Père Noël,cette année com
587     587     130205    Bonjour petit papa noël, ma so
588     588     98861     Bonjour Père Noël ,Comme vous 
589     589     39250     Je désire la paix dans le mond
590     590     65068     Cher Père Noël,je veut une voi
591     591     99083     Bonjour Père Noël  J'ai été tr
592     592     99299     Cher Pére Noël,J'ai été très s
593     593     101838    Boujour père Noël. Je trouve q
594     594     65072     Cher Père noël, j'ai mieux tra
595     595     114354    Cher Père Noël , bonjour je m'
596     596     121062    Très cher Papa Noël,Tout d'abo
597     597     43970     CHER 

1161    1161    101119    Chèr Père Noël,Je t'envoie cet
1162    1162    104496    Je voudrais Père Noël quelque 
1163    1163    98421     Bonjour papa noël, tu me recon
1164    1164    101217    Coucou Père Noël,merci papa no
1165    1165    119594    Allo cher Père Noël vous savez
1166    1166    99808     Bonjour Père Noël... je mapell
1167    1167    99784     Cher petit papa noël, mes pare
1168    1168    117927    Cher Père Noël,Je t'écris pour
1169    1169    53046     Salut Père Noël, même si j'ai 
1170    1170    64544     Bonjour Père Noël il me tarde 
1171    1171    64567     Cher père-noëlje voudrais t'éc
1172    1172    121766    Cher Père Noël,Maman m'aide un
1173    1173    107711    CHER Père Noël,J'AI ETE TRES S
1174    1174    56223     Cher Père Noël,Je sais que Noë
1175    1175    65073     Bonjour  Père Noël je veux le 
1176    1176    64968     Cher Père Noël,j'ai un frère q
1177    1177    107704    J'ai été très sage et j'ai bie
1178    1178    104482    Je vo

In [14]:
# SUPPRIMER LA TABLE

#conn=connexion_DB("santa_data")
#cursor = conn.cursor()

#rq_select='''DROP TABLE T_contenus
#            ;'''
    
#cursor.execute(rq_select)
#conn.commit()

#cursor.close()
#deconnexion_DB(conn)

PostgreSQL connection is closed
